In [1]:
# %%
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.stats import pearsonr, spearmanr
import h3pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
import arviz as az
import pymc as pm
import pymc.sampling.jax as pmjax
import jax

from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import force_console_behavior
master_bar, progress_bar = force_console_behavior()
import pytensor.tensor as pt
import re
from warnings import filterwarnings
import matplotlib.colors as colors
from sklearn.preprocessing import LabelEncoder
import geopandas as gpd 
import sys
import os

filterwarnings('ignore')
tqdm.pandas()

from utils import *

target_season = 'spring'


In [2]:
with open('../trophic_niches.txt', 'r') as f:
    niches = f.read().strip().split('\n')
    
niches += ['peak_IRG']
    

In [3]:
all_data = []
for target_niche in niches:
    data = pd.read_csv(f'../../data/D03.smoothed_peaks/smoothed_{target_niche}_{target_season}.csv')
    all_data.append(data)
all_data = pd.concat(all_data, axis=0).reset_index(drop=True)

In [7]:
if 'Unnamed: 0' in all_data.columns:
    del all_data['Unnamed: 0']

In [8]:
all_data.to_csv(f'../../data/D03.smoothed_peaks/all_smoothed_peaks_{target_season}.csv', index=False)

In [9]:
all_data


,h3_02,mean_DOY_max,std_DOY_max,mean_DOY_peak,std_DOY_peak,lng,lat,source_model,season,year,niche_or_level,mean_seasonality,geometry,cell_index,year_index,mu_ARR,std_ARR
0,82029ffffffffff,106.400000,11.067972,99.840000,22.584434,-129.851595,70.836600,10_40_20_50,spring,2010,all,NaN,POLYGON ((-132.86398054754267 72.0703696511523...,63,0,98.515795,25.634161
1,820837fffffffff,136.941176,46.774081,92.357143,34.331946,6.840723,63.444103,10_40_20_50,spring,2010,all,NaN,"POLYGON ((7.646201116757439 64.66507030510411,...",188,0,91.563741,36.562363
2,820887fffffffff,165.983607,38.409414,67.422222,11.580722,18.265517,60.172836,10_40_20_50,spring,2010,all,NaN,"POLYGON ((16.21328542634604 61.10408688097463,...",189,0,73.645659,14.810984
3,820897fffffffff,164.285714,48.196522,40.500000,20.506097,19.237456,57.931110,10_40_20_50,spring,2010,all,NaN,"POLYGON ((17.28620003196557 58.92634320829851,...",191,0,68.983216,22.892866
4,8208a7fffffffff,135.438931,38.525134,95.827434,25.173202,13.222211,60.705183,10_40_20_50,spring,2010,all,NaN,"POLYGON ((11.080660058482366 61.5405146000252,...",193,0,87.620640,27.825177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189620,82829ffffffffff,NaN,NaN,82.825073,17.455834,13.898732,4.279475,NaN,spring,2020,peak_IRG,1429.971436,POLYGON ((14.807652062069575 3.046341729325485...,2771,10,85.496585,21.824408
189621,8282c7fffffffff,NaN,NaN,89.250000,26.762847,6.978839,1.835551,NaN,spring,2020,peak_IRG,1056.394775,POLYGON ((7.786228442487397 0.6114455260916615...,2772,10,87.814826,32.288700
189622,8282d7fffffffff,NaN,NaN,79.347305,6.178010,9.187838,3.530667,NaN,spring,2020,peak_IRG,1431.231689,POLYGON ((10.026444557704739 2.318989904044636...,2774,10,80.118685,8.517562
189623,8282dffffffffff,NaN,NaN,81.463542,7.290439,6.787301,4.465236,NaN,spring,2020,peak_IRG,1001.054016,"POLYGON ((7.656464521877464 3.293008662639565,...",2775,10,82.653740,9.918451
